In [5]:
from pyspark.sql import SparkSession
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("tim-wywiol-intro")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

22/02/18 17:07:46 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [17]:
# Loading Data in SPARK RDD from HDFS
rdd_lines_en = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.en")
rdd_lines_de = spark_context.textFile("hdfs://192.168.2.119:9000/europarl/europarl-v7.de-en.de")


In [18]:
## A1 
## Count lines
print(rdd_lines_en.first())
print(rdd_lines_en.getNumPartitions())
print(rdd_lines_en.count())
print(rdd_lines_de.first())
print(rdd_lines_de.getNumPartitions())
print(rdd_lines_de.count())

Resumption of the session
3


1920209
Wiederaufnahme der Sitzungsperiode
3


1920209


In [25]:
## A2
#LowerCase RDD Text
def TokLow(line):
    new_line = line.lower()
    return new_line.split()
rdd_lines_en_low = rdd_lines_en.map(lambda x: TokLow(x))
rdd_lines_de_low = rdd_lines_de.map(lambda x: TokLow(x))

In [39]:
print(rdd_lines_en_low.take(2))
print(rdd_lines_en_low.count())
print(rdd_lines_de_low.take(2))
print(rdd_lines_de_low.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']]


1920209


[['wiederaufnahme', 'der', 'sitzungsperiode'], ['ich', 'erkläre', 'die', 'am', 'freitag,', 'dem', '17.', 'dezember', 'unterbrochene', 'sitzungsperiode', 'des', 'europäischen', 'parlaments', 'für', 'wiederaufgenommen,', 'wünsche', 'ihnen', 'nochmals', 'alles', 'gute', 'zum', 'jahreswechsel', 'und', 'hoffe,', 'daß', 'sie', 'schöne', 'ferien', 'hatten.']]


1920209


In [32]:
## A3
# Count 10 most frequent words
# Map
all_words_en = rdd_lines_en_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
print(all_words_en.take(20))


[('resumption', 1), ('of', 1), ('the', 1), ('session', 1), ('i', 1), ('declare', 1), ('resumed', 1), ('the', 1), ('session', 1), ('of', 1), ('the', 1), ('european', 1), ('parliament', 1), ('adjourned', 1), ('on', 1), ('friday', 1), ('17', 1), ('december', 1), ('1999,', 1), ('and', 1)]


In [37]:
from operator import add
all_words_counts_en = all_words_en.reduceByKey(add)
print(all_words_counts_en.takeOrdered(10, key=lambda x: -x[1]))

[('the', 3663317), ('of', 1737096), ('to', 1611846), ('and', 1345289), ('in', 1134117), ('that', 835926), ('a', 810820), ('is', 792599), ('for', 557377), ('we', 551272)]


In [38]:
## A3 Deutsch
all_words_de = rdd_lines_de_low\
                                .flatMap(lambda w: w)\
                                .map(lambda l: (l,1))
all_words_counts_de = all_words_de.reduceByKey(add)
print(all_words_counts_de.takeOrdered(10, key=lambda x: -x[1]))

[('die', 1980588), ('der', 1710400), ('und', 1338008), ('in', 781398), ('zu', 618887), ('den', 577690), ('wir', 489054), ('für', 478353), ('ich', 469069), ('das', 466167)]


In [44]:
#######  A4   #################################################################
#1 key line ((1, [w1, w2, w3]), (2, [w1,w2,w3])...
en_1  = rdd_lines_en_low.zipWithIndex()
#[(['resumption', 'of', 'the', 'session'], 0),
en_2 = en_1.map(lambda x: (x[1], x[0]))
print(en_2.take(3))

[(0, ['resumption', 'of', 'the', 'session']), (1, ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.']), (2, ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'])]


In [45]:
de_1  = rdd_lines_de_low.zipWithIndex()
de_2 = de_1.map(lambda x: (x[1], x[0]))

In [49]:
#4.3 Join RDDs
join_3 = en_2.join(de_2)
print(join_3.take(3))

22/02/18 18:27:00 ERROR TaskSetManager: Task 3 in stage 51.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 51.0 failed 4 times, most recent failure: Lost task 3.3 in stage 51.0 (TID 128) (192.168.2.119 executor 112): java.nio.file.FileSystemException: /tmp/spark-5b431f91-1505-4352-9b17-39a000a59416/executor-7fa94fb7-eac3-481f-855e-098ae8448c14/blockmgr-c895609c-7675-45d2-b033-6390f7503a38/1f: No space left on device
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:91)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:107)
	at sun.nio.fs.UnixFileSystemProvider.createDirectory(UnixFileSystemProvider.java:384)
	at java.nio.file.Files.createDirectory(Files.java:674)
	at org.apache.spark.storage.DiskBlockManager.getFile(DiskBlockManager.scala:96)
	at org.apache.spark.storage.DiskBlockManager.getFi

In [4]:
# release the cores for another application!
spark_context.stop()